In [21]:
import pandas as pd
import skmob
import geopandas as gpd
from skmob.preprocessing import *
from skmob.measures.individual import *
from skmob.preprocessing import detection
from preprocess_utils import *

### Preprocessing parameters

In [23]:
dataset_path = '../data/MilanoData.csv'
max_speed_kmh = 270
spatial_radius_km_compress = 0.02 # 20 meters
spatial_radius_km_stops = 0.100
minutes_for_a_stop = 20

shape_path = "../data/milan_medium.geojson"

#only the hour of the day is important
lower_time = pd.to_datetime("1994-10-14 15:00:00").time()
upper_time = pd.to_datetime("1994-10-14 16:00:00").time()

### 1. Data Loading

In [6]:
#loading the dataset

df_traj = pd.read_csv(dataset_path, sep=',', parse_dates=['datetime'])
df_traj[:2]

,userid,datetime,lat,lon
0,1059,2007-04-05 07:47:02,45.474777,9.210948
1,1059,2007-04-05 08:16:00,45.474777,9.210948


In [7]:
# print some statistics
print("# of users: "+str(len(df_traj['userid'].unique())))
print("# of points: "+str(len(df_traj)))
print("from: "+str(df_traj['datetime'].min()))
print("to: "+str(df_traj['datetime'].max()))

# of users: 17087
# of points: 1806293
from: 2007-04-01 00:00:02
to: 2007-04-07 23:59:25


Convert the `DataFrame` into a `TrajDataFrame`

In [9]:
tdf_traj = skmob.TrajDataFrame(df_traj, latitude='lat', longitude='lon', 
                          datetime='datetime', user_id='userid')

tdf_traj = tdf_traj.sort_by_uid_and_datetime()

### 2. Data Cleaning

Execution time: $\approx 2min$

In [10]:
%%time

tdf_filtered = skmob.preprocessing.filtering.filter(tdf_traj, max_speed_kmh=max_speed_kmh, 
                                    include_loops=False)

print("Filtered "+str(len(tdf_traj)-len(tdf_filtered))+" points.")
print(len(tdf_filtered))

Filtered 2 points.
1806291
Wall time: 1min 53s


Merge together all points that are closer than spatial_radius_km=0.05 kilometers from each other.<br>
Execution time: $\approx 3 min$

In [13]:
%%time

tdf_compressed = compression.compress(tdf_filtered, 
                                      spatial_radius_km=spatial_radius_km_compress)

print("Compressed "+str(len(tdf_filtered)-len(tdf_compressed))+" points.")
print("Radius: "+str(spatial_radius_km_compress))

Compressed 105941 points.
Radius: 0.02
Wall time: 3min 51s


### 3. Trajectory segmentation with Stop detection

#### Stop detection

A stop is detected when the individual spends at least minutes_for_a_stop minutes within a distance stop_radius_factor * spatial_radius km from a given trajectory point. The stop’s coordinates are the median latitude and longitude values of the points found within the specified distance <br>
Parameters: <br>
`minutes_for_a_stop = 20.0` <br>
`spatial_radius_km = 0.1`<br><br>
Execution time: $\approx 2min$

In [14]:
%%time

stdf = detection.stay_locations(tdf_traj, stop_radius_factor=None, 
                           minutes_for_a_stop=minutes_for_a_stop, 
                       spatial_radius_km=spatial_radius_km_stops, leaving_time=True)

Wall time: 2min 34s


#### Trajectory segmentation
Execution time: $\approx 4 min$

In [16]:
%%time

traj_seg = split_trajectories_in_tdf(tdf_compressed, stdf)

# create an UNIQUE traj_id as uid+'_'+tid
traj_ids = []
for uid, tid in zip(traj_seg['uid'], traj_seg['tid']):
    traj_ids.append(str(uid)+"_"+str(tid))

traj_seg = traj_seg.drop("tid", axis=1)
traj_seg['traj_id'] = traj_ids

Wall time: 8min


In [19]:
# print some statistics
print("# of users: "+str(len(traj_seg['uid'].unique())))
print("# of points: "+str(len(traj_seg)))
print("from: "+str(traj_seg['datetime'].min()))
print("to: "+str(traj_seg['datetime'].max()))

# of users: 15090
# of points: 1593114
from: 2007-04-01 00:00:02
to: 2007-04-07 23:59:25


### 4. Filter by geographic area

In [24]:
milan_medium = gpd.read_file(shape_path)

In [25]:
from skmob.utils.plot import *
# style of the tessellation
tex_style = {'fillColor':'blue', 'color':'black', 'opacity': 0.2}
plot_gdf(milan_medium, style_func_args=tex_style, zoom=10, popup_features=['tile_ID'])

Keep only the trajectories with at least one GPS points inside the geographic area of interest.

In [28]:
%%time

id_all_in, id_atleast_one_in = filter_in_shape(traj_seg, milan_medium, drop=False)

C:\Users\giuli\Anaconda3\envs\sk_dev\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


Wall time: 2min 1s


In [30]:
#take the trajectories with at least TWO points inside the region

traj_inside = traj_seg[traj_seg['uid'].isin(id_atleast_one_in)]
gb = traj_inside.groupby("uid", as_index=False).count()

ids_traj_mobility = gb[gb['lat']>1]['uid']
traj_filtered_area = traj_inside[traj_inside['uid'].isin(ids_traj_mobility)]

print(len(traj_filtered_area['uid'].unique()))

7797


### 5. Trajectory segmentation$^2$

Cut the trajectories wrt to the boundaries


In [35]:
%%time

res = segment_trajectories_area(traj_filtered_area)

traj_segmented_2 = traj_filtered_area.drop(['uid'], axis=1)
traj_segmented_2['uid'] = res

Wall time: 1.56 s


Filter 1. Keep only the sub-trajectories INSIDE the geographic region

In [36]:
ids_filter1 = list(traj_segmented_2[traj_segmented_2['isin']==True]['uid'].unique())
df_traj_f1 = traj_segmented_2[traj_segmented_2['uid'].isin(ids_filter1)]
print("Trajectories: "+str(len(ids_filter1)))

Trajectories: 32192


Filter 2. Keep only the sub-trajectories with at least 2 GPS points

In [38]:
gb = df_traj_f1.groupby("uid", as_index=False).count()
ids_filter2 = list(gb[gb['isin']>1].uid.unique())
df_traj_f2 = df_traj_f1[df_traj_f1['uid'].isin(ids_filter2)]
print("Trajectories: "+str(len(ids_filter2)))

Trajectories: 27149


Filter 3. Keep only working days

In [40]:
trips_grouped = df_traj_f2.groupby(['uid'], as_index=False).first()
trips_grouped['day_number'] = trips_grouped['datetime'].apply(lambda d: d.weekday())
trips_grouped[:2]

,uid,datetime,lat,lng,traj_id,points,isin,day_number
0,100000_16000,2007-04-01 13:27:53,45.481099,9.205389,100000_2,POINT (9.20539 45.48110),True,6
1,100000_16002,2007-04-01 18:12:54,45.487850,9.157280,100000_3,POINT (9.15728 45.48785),True,6


In [42]:
tid_2_keep = trips_grouped[trips_grouped['day_number']<5]['uid'].unique()
traj_d_week = df_traj_f2[df_traj_f2['uid'].isin(tid_2_keep)]
print(len(traj_d_week['uid'].unique()))

21037


### 6. Save the pre-processed dataset

In [47]:
traj_d_week.to_csv("../data/milan_dataset_preprocessed.csv", index=False)